<a href="https://colab.research.google.com/github/Tanmay240405/disease-detection/blob/main/CNNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tanmaysaxena2404","key":"ff86394d4447e0dc917ac4fec905f3ae"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

# STEP 4: Unzip the dataset into a folder called "data"
!unzip -q breast-histopathology-images.zip -d data

# STEP 5: Verify contents
!ls data

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/breast-histopathology-images
License(s): CC0-1.0
 99% 3.08G/3.10G [00:27<00:00, 112MB/s] 
100% 3.10G/3.10G [00:27<00:00, 121MB/s]
10253  10301  12872  12930  13613  14305  16554  9041  9259
10254  10302  12873  12931  13616  14306  16555  9043  9260
10255  10303  12875  12932  13617  14321  16568  9044  9261
10256  10304  12876  12933  13666  15471  16569  9073  9262
10257  10305  12877  12934  13687  15472  16570  9075  9265
10258  10306  12878  12935  13688  15473  16895  9076  9266
10259  10307  12879  12947  13689  15510  16896  9077  9267
10260  10308  12880  12948  13691  15512  8863	 9078  9290
10261  12241  12881  12949  13692  15513  8864	 9081  9291
10262  12242  12882  12951  13693  15514  8865	 9083  9319
10264  12626  12883  12954  13694  15515  8867	 9123  9320
10268  12748  12884  12955  13916  15516  8913	 9124  9321
10269  12749  12886  13018  14078  15632  8914	 9125  9322
10272  12750  12890  13019  14079

In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D
from keras.layers import Flatten, MaxPool2D, MaxPooling2D, BatchNormalization

In [ ]:
from glob import glob

# Path to unzipped dataset
main_path = "data"  # this is where you unzipped

# Get all PNG images recursively
imagePatches = glob(main_path + "/**/*.png", recursive=True)

# Print first 10 image paths
for filename in imagePatches[:10]:
    print(filename)

# Total images
print("Total images found:", len(imagePatches))


data/12880/1/12880_idx5_x401_y1001_class1.png
data/12880/1/12880_idx5_x651_y2051_class1.png
data/12880/1/12880_idx5_x651_y951_class1.png
data/12880/1/12880_idx5_x1801_y1901_class1.png
data/12880/1/12880_idx5_x1151_y751_class1.png
data/12880/1/12880_idx5_x901_y1051_class1.png
data/12880/1/12880_idx5_x2301_y1701_class1.png
data/12880/1/12880_idx5_x1501_y801_class1.png
data/12880/1/12880_idx5_x1501_y1101_class1.png
data/12880/1/12880_idx5_x1651_y751_class1.png
Total images found: 289159


In [ ]:
class0 = []
class1 = []
for filename in imagePatches:
    if filename.endswith("class0.png"):
        class0.append(filename)
    else:
        class1.append(filename)
import random
sampled_class0 = random.sample(class0, 78786)
sampled_class1 = random.sample(class1, 78786)
print(len(class0))
print(len(class1))
print(len(sampled_class1))

207914
81245
78786


In [ ]:
from matplotlib.image import imread
import cv2

In [ ]:
def get_image_array(data, label):
  img_arrays = []
  for i in data:
    if i.endswith(".png"):
      img = cv2.imread(i, cv2.IMREAD_COLOR)
      img_sized = cv2.resize(img, (50,50), interpolation=cv2.INTER_LINEAR)
      img_arrays.append([img_sized, label])
  return img_arrays

class0_array = get_image_array(sampled_class0, 0)
class1_array = get_image_array(sampled_class1, 1)

In [ ]:
test = cv2.imread("data/8863/1/8863_idx5_x1001_y801_class1.png")
print(test.shape)

(50, 50, 3)


In [ ]:

combine_data = class0_array + class1_array

import random
random.seed(42)
random.shuffle(combine_data)

print(type(combine_data[0]))
print(combine_data[0][0].shape)
print(combine_data[0][1])
print("Total samples:", len(combine_data))


<class 'list'>
(50, 50, 3)
1
Total samples: 157572


In [ ]:
x = []
y = []
for features, label in combine_data:
  x.append(features)
  y.append(label)

In [ ]:
x = np.array(x).reshape(-1, 50, 50, 3)
x.shape

(157572, 50, 50, 3)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(118179, 50, 50, 3) (39393, 50, 50, 3) (118179, 2) (39393, 2)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam

def create_model():
  model = Sequential()
  model.add(Conv2D(64, kernel_size = 3, activation = "relu", input_shape = (50, 50, 3)))
  model.add(Conv2D(64, kernel_size = 3, activation = "relu"))
  model.add(Flatten())
  model.add(Dense(2, activation = "softmax"))
  adam = Adam(learning_rate = 0.0001)
  model.compile(loss = "binary_crossentropy", optimizer = adam, metrics = ["accuracy"])
  return model

model = create_model()

history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=16, epochs=20, verbose=2)

Epoch 1/20


KeyboardInterrupt: 